In [1]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from dotenv import load_dotenv

load_dotenv()

model = ChatOpenAI(model="gpt-4o-mini") #평가시 사용할 llm 모델은 성능 좋은 것을 써야 좀 더 정확한 평가가 가능.
embedding_model = OpenAIEmbeddings(
    model="text-embedding-3-small"
)


In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma

COLLECTION_NAME = "medicine_docs"
PERSIST_DIRECTORY = "vector_store/chroma/medicine_db2"

path = r'data/medicine.txt'
loader = TextLoader(path, encoding='utf-8')

splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    model_name="gpt-4o-mini",
    chunk_size=500,
    chunk_overlap=100
)
docs = loader.load_and_split(splitter)

vector_store = Chroma.from_documents(
    documents=docs,
    embedding=embedding_model,
    collection_name=COLLECTION_NAME,
    persist_directory=PERSIST_DIRECTORY
)

In [3]:
# from langchain_chroma import Chroma

# COLLECTION_NAME = "medicine_docs"
# PERSIST_DIRECTORY = "vector_store/chroma/medicine_db2"
# EMBEDDING_MODEL_NAME = 'text-embedding-3-small'

# vector_store = Chroma(
#     embedding_function=EMBEDDING_MODEL_NAME,
#     collection_name=COLLECTION_NAME,
#     persist_directory=PERSIST_DIRECTORY
# )

In [4]:
vector_store._collection.count()

21336

In [5]:
# RAG 체인 구성
# query -> pt -> llm -> 응답 (query에 같이 입력된 context, 답변 => 2가지를 출력)
from langchain import hub
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_core.documents import Document
from operator import itemgetter

# prompt template. langchain hub에 등록된 것을 가져와서 사용.
prompt_template = hub.pull("rlm/rag-prompt")
# prompt_template

# Retriever 생성
retriever = vector_store.as_retriever()


def format_docs(src_docs:dict[str, list[Document]]) -> str:
    """list[Document]: Vector Store에서 검색한 context들에서 
    page_content만 추출해서 하나의 문자열로 합쳐서 반환"""
    docs = src_docs['context']
    return "\n\n".join([doc.page_content for doc in docs])

def str_from_documents(docs: list[Document]) -> list[str]:
    """list[Document]에서 page_content 값들만 추출한 list를 반환."""
    return [doc.page_content for doc in docs]

rag_chain = (
    RunnablePassthrough() # rag chain을 RunnableSequence로 만들기 위해 Runnable인 것으로 시작.
    | {
        "context": retriever, "question":RunnablePassthrough()
    } # retriver -> {"context":list[Document], "question":"user input"}
    | {
        # 앞에서 넘어온 dictionary에서 context(List[Document])를 추출 -> page_content값들을 list로 반환. list[str]
        "source_context" : itemgetter("context") | RunnableLambda(str_from_documents), 
        "llm_answer": {
            # {"context":list[Document]} -> str(page_content들만 모은 string)
            "context": RunnableLambda(format_docs), "question":itemgetter("question")
        } | prompt_template | model | StrOutputParser()  # LLM 응답 처리 chain. 
    }
)

In [6]:
retriever

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x0000026770848050>, search_kwargs={})

In [7]:
user_input = "이명에 관한 약 추천해줘줘."
response = rag_chain.invoke(user_input)
# response: dictionary - {source_context:VectorStore에 조회한 context들, llm_answer:LLM 답변변}

In [8]:
print(type(response))
print(response.keys())
print("답변:", response['llm_answer'])
response['source_context']

<class 'dict'>
dict_keys(['source_context', 'llm_answer'])
답변: 이명에 대한 특정 약물 정보는 제공되지 않았습니다. 비아핀에멀젼(트롤아민)은 화상 및 비감염성 피부상처에 사용되는 약물입니다. 이명에 대한 치료는 의사와 상담하여 적절한 약물을 처방받는 것이 좋습니다.


['비아핀에멀젼(트롤아민)\n고려제약(주)\n이 약은 1도, 2도 화상 및 기타 비감염성 피부상처, 방사선 치료에 의한 2차적 홍반(붉은 반점)에 사용합니다.\n1도 화상인 경우 1일 2~4회 약물이 더 이상 흡수되지 않을 때까지 두껍게 바른 후 부드럽게 마사지합니다.2도 화상 및 기타 비감염성 피부상처인 경우 상처부위와 그 주위에 약물을 두껍게 바르고 항상 상처부위에 약물이 과량 존재하도록 반복해서 바르며 드레싱이 필요한 경우에는 이 약을 두껍게 바른 후(약 0.5 cm 두께로) 축축한 무균패드로 덮고 드레싱 합니다.방사선 치료에 의한 2차적 홍반(붉은 반점)인 경우 일정한 간격을 두고 1일 2~3회 방사선을 조사할 부위와 주위에 약물을 두껍게 바르고 부드럽게 마사지합니다.\n이 약에 과민증 환자 또는 경험자,\xa0약물이나 음식 알레르기에 의한 피부염,\xa0출혈성 및 감염성 피부상처 부위, 감염 부위, 이식을 행하기 전의 피부이식 부위에는 이 약을 사용하지 마십시오.이 약을 사용하기 전에\xa0프리필렌글리콜에 과민증 환자 또는 경험자는\xa0의사 또는 약사와 상의하십시오.이 약은 살균제를 함유하지 않으므로 상처부위를 청결하게 유지하십시오.정해진 용법과 용량을 잘 지키십시오.안과용으로 사용하지 않고, 만일 눈에 들어간 경우 물로 씻어내십시오.이 약은 일광(햇볕)차단제를 함유하지 않으므로 일광(햇볕)화상 방지용으로 사용하지 마십시오.\n실온에서 보관하십시오.',
 '비아핀에멀젼(트롤아민)\n고려제약(주)\n이 약은 1도, 2도 화상 및 기타 비감염성 피부상처, 방사선 치료에 의한 2차적 홍반(붉은 반점)에 사용합니다.\n1도 화상인 경우 1일 2~4회 약물이 더 이상 흡수되지 않을 때까지 두껍게 바른 후 부드럽게 마사지합니다.2도 화상 및 기타 비감염성 피부상처인 경우 상처부위와 그 주위에 약물을 두껍게 바르고 항상 상처부위에 약물이 과량 존재하도록 반복해서 바르며 드레싱이 필요한 경우에는 이 약을 두껍게 바른 후(약 0.5 cm 두께로) 축축한 무균패드로 

In [9]:
user_input = "이명에 관한 약 추천해줘."
answer = response['llm_answer']
context_list = response['source_context']
ground_truth = "이명에 관한 약은 이티민정40밀리그램, 써큐프리정, 징코브이연질캡슐 등이 있습니다."
print(user_input) # 질문
print(answer)     # llm 모델 답변
print(context_list) # vector store에서 조회한 context들.
print(ground_truth) # 정답

이명에 관한 약 추천해줘.
이명에 대한 특정 약물 정보는 제공되지 않았습니다. 비아핀에멀젼(트롤아민)은 화상 및 비감염성 피부상처에 사용되는 약물입니다. 이명에 대한 치료는 의사와 상담하여 적절한 약물을 처방받는 것이 좋습니다.
['비아핀에멀젼(트롤아민)\n고려제약(주)\n이 약은 1도, 2도 화상 및 기타 비감염성 피부상처, 방사선 치료에 의한 2차적 홍반(붉은 반점)에 사용합니다.\n1도 화상인 경우 1일 2~4회 약물이 더 이상 흡수되지 않을 때까지 두껍게 바른 후 부드럽게 마사지합니다.2도 화상 및 기타 비감염성 피부상처인 경우 상처부위와 그 주위에 약물을 두껍게 바르고 항상 상처부위에 약물이 과량 존재하도록 반복해서 바르며 드레싱이 필요한 경우에는 이 약을 두껍게 바른 후(약 0.5 cm 두께로) 축축한 무균패드로 덮고 드레싱 합니다.방사선 치료에 의한 2차적 홍반(붉은 반점)인 경우 일정한 간격을 두고 1일 2~3회 방사선을 조사할 부위와 주위에 약물을 두껍게 바르고 부드럽게 마사지합니다.\n이 약에 과민증 환자 또는 경험자,\xa0약물이나 음식 알레르기에 의한 피부염,\xa0출혈성 및 감염성 피부상처 부위, 감염 부위, 이식을 행하기 전의 피부이식 부위에는 이 약을 사용하지 마십시오.이 약을 사용하기 전에\xa0프리필렌글리콜에 과민증 환자 또는 경험자는\xa0의사 또는 약사와 상의하십시오.이 약은 살균제를 함유하지 않으므로 상처부위를 청결하게 유지하십시오.정해진 용법과 용량을 잘 지키십시오.안과용으로 사용하지 않고, 만일 눈에 들어간 경우 물로 씻어내십시오.이 약은 일광(햇볕)차단제를 함유하지 않으므로 일광(햇볕)화상 방지용으로 사용하지 마십시오.\n실온에서 보관하십시오.', '비아핀에멀젼(트롤아민)\n고려제약(주)\n이 약은 1도, 2도 화상 및 기타 비감염성 피부상처, 방사선 치료에 의한 2차적 홍반(붉은 반점)에 사용합니다.\n1도 화상인 경우 1일 2~4회 약물이 더 이상 흡수되지 않을 때까지 두껍게 바른 후 부드럽게 마사지합니다.2도 화상

In [10]:
from ragas import SingleTurnSample, EvaluationDataset
# 평가 데이터셋을 구성.

## 개별 평가 데이터
sample = SingleTurnSample(
    user_input=user_input,           # 사용자 입력-질문
    retrieved_contexts=context_list, # 질문에 대해서 Vector Store에서 조회한 context들.
    response=answer,                 # LLM 답변 (정답 추정값)
    reference=ground_truth           # 정답 답변변
)

# 평가 데이터셋을 구성
eval_dataset = EvaluationDataset(samples=[sample])
print(type(eval_dataset))
eval_dataset

<class 'ragas.dataset_schema.EvaluationDataset'>


EvaluationDataset(features=['user_input', 'retrieved_contexts', 'response', 'reference'], len=1)

In [11]:
eval_dataset[0]

SingleTurnSample(user_input='이명에 관한 약 추천해줘.', retrieved_contexts=['비아핀에멀젼(트롤아민)\n고려제약(주)\n이 약은 1도, 2도 화상 및 기타 비감염성 피부상처, 방사선 치료에 의한 2차적 홍반(붉은 반점)에 사용합니다.\n1도 화상인 경우 1일 2~4회 약물이 더 이상 흡수되지 않을 때까지 두껍게 바른 후 부드럽게 마사지합니다.2도 화상 및 기타 비감염성 피부상처인 경우 상처부위와 그 주위에 약물을 두껍게 바르고 항상 상처부위에 약물이 과량 존재하도록 반복해서 바르며 드레싱이 필요한 경우에는 이 약을 두껍게 바른 후(약 0.5 cm 두께로) 축축한 무균패드로 덮고 드레싱 합니다.방사선 치료에 의한 2차적 홍반(붉은 반점)인 경우 일정한 간격을 두고 1일 2~3회 방사선을 조사할 부위와 주위에 약물을 두껍게 바르고 부드럽게 마사지합니다.\n이 약에 과민증 환자 또는 경험자,\xa0약물이나 음식 알레르기에 의한 피부염,\xa0출혈성 및 감염성 피부상처 부위, 감염 부위, 이식을 행하기 전의 피부이식 부위에는 이 약을 사용하지 마십시오.이 약을 사용하기 전에\xa0프리필렌글리콜에 과민증 환자 또는 경험자는\xa0의사 또는 약사와 상의하십시오.이 약은 살균제를 함유하지 않으므로 상처부위를 청결하게 유지하십시오.정해진 용법과 용량을 잘 지키십시오.안과용으로 사용하지 않고, 만일 눈에 들어간 경우 물로 씻어내십시오.이 약은 일광(햇볕)차단제를 함유하지 않으므로 일광(햇볕)화상 방지용으로 사용하지 마십시오.\n실온에서 보관하십시오.', '비아핀에멀젼(트롤아민)\n고려제약(주)\n이 약은 1도, 2도 화상 및 기타 비감염성 피부상처, 방사선 치료에 의한 2차적 홍반(붉은 반점)에 사용합니다.\n1도 화상인 경우 1일 2~4회 약물이 더 이상 흡수되지 않을 때까지 두껍게 바른 후 부드럽게 마사지합니다.2도 화상 및 기타 비감염성 피부상처인 경우 상처부위와 그 주위에 약물을 두껍게 바르고 항상 상처부위에 약물이 과량 존재하

In [12]:
# Dataset을 Pandas DataFrame으로 변환
eval_df = eval_dataset.to_pandas()
eval_df

,user_input,retrieved_contexts,response,reference
0,이명에 관한 약 추천해줘.,"[비아핀에멀젼(트롤아민)\n고려제약(주)\n이 약은 1도, 2도 화상 및 기타 비감...",이명에 대한 특정 약물 정보는 제공되지 않았습니다. 비아핀에멀젼(트롤아민)은 화상 ...,"이명에 관한 약은 이티민정40밀리그램, 써큐프리정, 징코브이연질캡슐 등이 있습니다."


In [13]:
# 평가

from ragas.metrics import (
    LLMContextRecall, Faithfulness, LLMContextPrecisionWithReference, AnswerRelevancy
)
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas import evaluate

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from dotenv import load_dotenv

load_dotenv()

model = ChatOpenAI(model="gpt-4o") # 평가시 사용할 llm 모델.
eval_llm = LangchainLLMWrapper(model) # Langchain 모델을 ragas에서 사용할 수있도록 변환.

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
eval_embedding = LangchainEmbeddingsWrapper(embedding_model)

In [14]:
#평가 지표들을 List로 묶는다.
metrics = [
    LLMContextRecall(llm=eval_llm),
    LLMContextPrecisionWithReference(llm=eval_llm),
    Faithfulness(llm=eval_llm),
    AnswerRelevancy(llm=eval_llm, embeddings=eval_embedding)
]
result = evaluate(dataset=eval_dataset, metrics=metrics)

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

In [15]:
type(result)
result

{'context_recall': 0.0000, 'llm_context_precision_with_reference': 0.0000, 'faithfulness': 0.6000, 'answer_relevancy': 0.0000}

In [16]:
result.to_pandas()

,user_input,retrieved_contexts,response,reference,context_recall,llm_context_precision_with_reference,faithfulness,answer_relevancy
0,이명에 관한 약 추천해줘.,"[비아핀에멀젼(트롤아민)\n고려제약(주)\n이 약은 1도, 2도 화상 및 기타 비감...",이명에 대한 특정 약물 정보는 제공되지 않았습니다. 비아핀에멀젼(트롤아민)은 화상 ...,"이명에 관한 약은 이티민정40밀리그램, 써큐프리정, 징코브이연질캡슐 등이 있습니다.",0.0,0.0,0.6,0.0


In [17]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = TextLoader(path, encoding='utf-8')
splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    model_name="gpt-4o-mini",
    chunk_size=500,
    chunk_overlap=100
)
docs = loader.load_and_split(splitter)
print(len(docs))

6501


In [18]:
# 평가 데이터로 사용할 context k개를 랜덤하게 추출
import random
total_samples = 5 # 추출할 샘플 개수
eval_context_list = [] # Sample들을 담을 리스트
while len(eval_context_list) < 5:
    _context = docs[random.randint(0, len(docs)-1)].page_content
    if len(_context) < 100: # 글자수 너무 적으면 질문을 생성 못하기 때문에 글자수 체크.
        continue
    eval_context_list.append(_context)

In [19]:
# Chain을 이용해서 LLM에게 질문-답 생성을 요청.
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from textwrap import dedent

# JSONOutputParser에서 사용할 스키마 생성.
class EvalDatasetSchema(BaseModel):
    user_input:str = Field(..., description="질문(Question)")
    retrieved_contexts:list[str] = Field(..., description="LLM이 답변할 때 참조할 context")
    reference: str = Field(..., description="정답(ground truth)")

parser = JsonOutputParser(pydantic_object=EvalDatasetSchema)
# print(parser.get_format_instructions())

prompt_template = PromptTemplate.from_template(
    template=dedent("""
        당신은 RAG 평가를 위해 질문과 정답 쌍을 생성하는 인공지능 비서입니다.
        다음 [Context] 에 문서가 주어지면 해당 문서를 기반으로 {num_questions}개의 질문을 생성하세요. 

        질문과 정답을 생성한 후 아래의 출력 형식 GUIDE 에 맞게 생성합니다.
        질문은 반드시 [context] 문서에 있는 정보를 바탕으로 생성해야 합니다. [context]에 없는 내용을 가지고 질문-답변을 절대 만들면 안됩니다.
        질문은 간결하게 작성합니다.
        하나의 질문에는 한 가지씩만 내용만 작성합니다. 
        질문을 만들 때 "제공된 문맥에서", "문서에 설명된 대로", "주어진 문서에 따라" 또는 이와 유사한 말을 하지 마세요.
        질문을 만들 때, 구체적인 증상에 따른 약을 추천받는 질문만 하세요.
        구체적인 증상에 따른 약 추천받는 질문만 하세요. 제발. 다른 질문은 하지 마세요.
        질문을 할 때는 구체적인 약 이름을 기반으로 질문하세요. "이 약" 또는 이와 유사한 말을 사용하지 마세요.
        
        정답은 반드시 [context]에 있는 정보를 바탕으로 작성합니다. 없는 내용을 추가하지 않습니다.
        정답은 반드시 [context]에 있는 구체적인 약 이름을 명시해서 답변하세요. 
        정답에는 구체적인 약 이름으로 추천해서 답변하세요.
        정답에 "이 약"이라는 말을 쓰지 마세요. "이 약"이라는 말을 쓰지 마. 제발. 쓰지 말라면 좀 쓰지마.
                    
        질문과 답변을 만들고 그 내용이 [context] 에 있는 항목인지 다시 한번 확인합니다.
        생성된 질문-답변 쌍은 반드시 dictionary 형태로 정의하고 list로 묶어서 반환해야 합니다.
        질문-답변 쌍은 반드시 {num_questions}개를 만들어 주십시오.
                    
        출력 형식: {format_instructions}

        [Context]
        {context}
        """
    ),
    partial_variables={"format_instructions":parser.get_format_instructions()}
)

model = ChatOpenAI(model="gpt-4o")
dataset_generator_chain = prompt_template | model | parser

In [20]:
c = eval_context_list[0]

qa = dataset_generator_chain.invoke({"context":c, "num_questions":5})

In [21]:
# 확인 후 retrieved_contexts을 context로 변경.
for d in qa:
    d['retrieved_contexts'] = [c]
qa

[{'user_input': '만성 간질환에 어떤 약을 추천하나요?',
  'retrieved_contexts': ['리버텍트골드연질캡슐\n한국코러스(주)\n이 약은 만성 간질환, 간경변, 독성 간질환의 보조치료에 사용합니다.\n성인은 1회 1캡슐 1일 3회 복용합니다.\n이 약에 과민증 환자, 대두유 또는 콩, 땅콩에 과민증 환자, 심한 담도폐쇄 환자, 12세 이하 소아는\xa0이 약을 복용하지 마십시오.이 약을 복용하기 전에 고지단백혈증, 당뇨병성 고지질혈증 및 췌장염 등 지방대사이상, 지방과부하로 특별한 위험이 예상되는 환자, 임부 또는 임신하고 있을 가능성이 있는 여성, 수유부는 의사 또는 약사와 상의하십시오.정해진 용법과 용량을 잘 지키십시오.\n습기와 빛을 피해 실온에서 보관하십시오.어린이의 손이 닿지 않는 곳에 보관하십시오.'],
  'reference': '만성 간질환에는 리버텍트골드연질캡슐을 추천합니다.'},
 {'user_input': '간경변 치료에 도움이 되는 약은 무엇인가요?',
  'retrieved_contexts': ['리버텍트골드연질캡슐\n한국코러스(주)\n이 약은 만성 간질환, 간경변, 독성 간질환의 보조치료에 사용합니다.\n성인은 1회 1캡슐 1일 3회 복용합니다.\n이 약에 과민증 환자, 대두유 또는 콩, 땅콩에 과민증 환자, 심한 담도폐쇄 환자, 12세 이하 소아는\xa0이 약을 복용하지 마십시오.이 약을 복용하기 전에 고지단백혈증, 당뇨병성 고지질혈증 및 췌장염 등 지방대사이상, 지방과부하로 특별한 위험이 예상되는 환자, 임부 또는 임신하고 있을 가능성이 있는 여성, 수유부는 의사 또는 약사와 상의하십시오.정해진 용법과 용량을 잘 지키십시오.\n습기와 빛을 피해 실온에서 보관하십시오.어린이의 손이 닿지 않는 곳에 보관하십시오.'],
  'reference': '간경변 치료에는 리버텍트골드연질캡슐을 사용할 수 있습니다.'},
 {'user_input': '독성 간질환에 적합한 약이 있나요?',
  'retrieved_con

In [22]:
import pandas as pd
pd.DataFrame(qa)

,user_input,retrieved_contexts,reference
0,만성 간질환에 어떤 약을 추천하나요?,"[리버텍트골드연질캡슐\n한국코러스(주)\n이 약은 만성 간질환, 간경변, 독성 간질...",만성 간질환에는 리버텍트골드연질캡슐을 추천합니다.
1,간경변 치료에 도움이 되는 약은 무엇인가요?,"[리버텍트골드연질캡슐\n한국코러스(주)\n이 약은 만성 간질환, 간경변, 독성 간질...",간경변 치료에는 리버텍트골드연질캡슐을 사용할 수 있습니다.
2,독성 간질환에 적합한 약이 있나요?,"[리버텍트골드연질캡슐\n한국코러스(주)\n이 약은 만성 간질환, 간경변, 독성 간질...",독성 간질환에는 리버텍트골드연질캡슐이 적합합니다.
3,12세 이하 소아가 복용하면 안 되는 약은 무엇인가요?,"[리버텍트골드연질캡슐\n한국코러스(주)\n이 약은 만성 간질환, 간경변, 독성 간질...",리버텍트골드연질캡슐은 12세 이하 소아가 복용하면 안 됩니다.
4,대두유에 과민증이 있는 사람이 피해야 할 약은?,"[리버텍트골드연질캡슐\n한국코러스(주)\n이 약은 만성 간질환, 간경변, 독성 간질...",대두유에 과민증이 있는 사람은 리버텍트골드연질캡슐을 피해야 합니다.
